# Mega-Sena

- **Aluno**: Álvaro Ferreira Pires de Paiva
- **Matrícula**: 2016039162

**Sumário**
- [Base de dados](#database)
- [Preparando o ambiente](#env_treatment)
  - Prêmio máximo
  - Rótulo
- [Análises](#analysis)
  - Dicas

## <a id="database">Base de dados</a>

Nessa avaliação será utilizado a base de dados referente aos concursos da Mega-Sena. 

A **Mega-Sena** tem sido a maior modalidade lotérica do Brasil, sendo uma entre as dez modalidades atuais das loterias da Caixa (Caixa Econômica Federal), com sorteios ordinários duas vezes por semana, além da Mega-Sena da Virada.

Para ganhar o **prêmio máximo da Mega-Sena** — **a sena** — , é necessário acertar a sena, o que significa obter coincidência entre **seis dos números** apostados e os seis números sorteados, de um total de sessenta dezenas (**de 01 a 60**), independentemente da ordem da aposta ou da ordem do sorteio. 

O concurso prevê também a chance de se ganhar parte do prêmio máximo, pelo acerto da **quina** (apenas cinco dos números sorteados), ou da **quadra** (apenas quatro dos números sorteados), com prêmios significativamente menores que aquele que seria pago na ocorrência do acerto da sena, o da quina maior que o da quadra. 

A base de dados está em formato de planilha em excel.

## <a id="env_treatment">Preparando o ambiente</a>
Aqui iremos realizar as importações de bibliotecas necessárias e o armazenamento dos dados para podermos analisarmos logo em seguida.

In [1]:
# Imports
import pandas as pd

In [2]:
# Leitura dos dados
excel_file = 'basedados.xlsx'
excel = pd.ExcelFile(excel_file)
data = excel.parse(0)

### Prêmio máximo
Para cada concurso da Mega-Sena, a maior premiação está relacionada com o acerto das 6 dezenas, e o seu valor é dada pela seguinte relação:

$$Prêmio = Ganhadores\_Sena	\times Rateio\_Sena$$

In [3]:
data['premio_maximo'] = data.Ganhadores_Sena * data.Rateio_Sena

### Rótulo
Criaremos uma nova coluna chamada "Rótulo" baseada na faixa de valores da coluna "prêmio_máximo", como apresentado na seguinte tabela:

| Intervalo [Milhões de Reais] | Rótulo |
|------------------------------|--------|
| 0 - 1                        | I      |
| 1 - 5                        | II     |
| 5 - 10                       | III    |
| 10 - 20                      | IV     |
| 20 - 40                      | V      |
| 40 - 80                      | VI     |
| 80 - 100                     | VII    |
| 100 - 200                    | VIII   |
| 200 - 300                    | IX     |
| 300 -                        | X      |

In [4]:
data['rotulo'] = pd.cut(data.premio_maximo,
                        bins=[0,1e6,5e6,1e7,2e7,4e7,8e7,1e8,2e8,3e8,1e9],
                        labels=["I","II","III","IV","V","VI","VII","VIII","IX","X"],
                        include_lowest=True)

## <a id="analysis">Análises</a>
Nessa seção, iremos realizar algumas análises nos dados que temos em mãos.

### Dicas
Pela Mega-Sena possuir prêmios de valores bastantes elevados, é normal encontrar sites com dicas para o jogador ter mais chances de ganhar. Sendo assim, iremos análisar 8 dicas encontradas em alguns sites e verificar a sua veracidade. As dicas são:

1. Os números com final 9 ou 0 saem pouco, por isso, é bom evitar escolher várias números com essa sequencia.
2. As dezenas 01, 02, 03, 11, 22, 44, 55, 48 e 57 saem pouco.
3. Não jogue números seguidos.
4. Não jogue em números que estejam na mesma linha vertical.
5. Divida a cartela em quatro quadrantes e distribua seu jogo entre eles.
6. Jogue sempre a mesma quantidade de dezenas pares e ímpares. Na Mega-Sena, 81% dos sorteios têm o seguinte esquema: 3 números pares e 3 ímpares ou 4 pares e 2 ímpares, ou vice-versa.
7. Prefira apenas um cartão com mais de seis dezenas do que vários cartões de seis dezenas. A regra é simples: quanto mais números você apostar, maiores são as chances de acertar um deles.
8. Um jogo em que se preenche 12 dezenas utilizando essas regras teria mais chances de êxito. Logo, o bolão é a melhor chance que se tem de ganhar.

Algumas variáveis iremos utilizar na validação de mais de uma dica, sendo elas:

In [5]:
# Colunas que iremos trabalhar
cols_dezenas = ['Dezena1', 'Dezena2', 'Dezena3', 'Dezena4', 'Dezena5', 'Dezena6']

# Dataframe filtrado pelas colunas que queremos
data_cols = data[cols_dezenas]

#### 1. Os números com final 9 ou 0 saem pouco

In [6]:
# Vetor com índices e 0 a 9 que usaremos para contar
count_last_digit = [0 for i in range(0, 10)]

# Iremos percorrer cada celula do dataframe e verificar sua terminação
for np_array in data_cols.values:
    for number in np_array:
        count_last_digit[ number % 10 ] += 1

In [7]:
# Total de números contados
last_digit_total = sum(count_last_digit)

# Ordenamos a lista pela quantidade de ocorrências de cada dígito (do menor para o maior)
last_digit_sort = sorted(enumerate(count_last_digit), key=lambda x:x[1])
# Invertemos a ordem para facilitar a visualização
last_digit_sort.reverse()

# Imprimimos a porcentagem de cada um
for index, value in last_digit_sort:
    print("{} - {}".format(index, (value*100/last_digit_total)))

3 - 10.576131687242798
4 - 10.444444444444445
2 - 10.148148148148149
0 - 10.08230452674897
7 - 9.958847736625515
8 - 9.893004115226338
1 - 9.868312757201647
6 - 9.76954732510288
5 - 9.670781893004115
9 - 9.588477366255145


Através das informações apresentadas logo acima, podemos constatar que:
- Números com último dígito sendo **0** possuem a quarta maior ocorrência nos sorteios até agora;
- Realmente os números com último dígito sendo **9** são os que menos saem nos sorteios. 

Sendo assim, a dica é verdade quando se trata apenas do dígito 9.

#### 2. As dezenas 01, 02, 03, 11, 22, 44, 55, 48 e 57 saem pouco

In [8]:
# Vetor com dígitos de 1 a 60
count_digit = [0 for i in range(0, 60)]

# Iremos percorrer cada celula do dataframe
for np_array in data_cols.values:
    for number in np_array:
        count_digit[ number-1 ] += 1

In [9]:
# Ordenamos a lista pela quantidade de ocorrências de cada dígito (do menor para o maior)
digit_sort = sorted(enumerate(count_digit), key=lambda x:x[1])
# Invertemos a ordem para facilitar a visualização
digit_sort.reverse()

In [10]:
find_digits = [1, 2, 3, 11, 22, 44, 55, 48, 57]
for digit, value in digit_sort:
    if digit in find_digits:
        print("Dezena: {} | Lugar no ranking de ocorrências: {}".format(digit, digit_sort.index((digit, value))))

Dezena: 22 | Lugar no ranking de ocorrências: 3
Dezena: 3 | Lugar no ranking de ocorrências: 4
Dezena: 1 | Lugar no ranking de ocorrências: 21
Dezena: 55 | Lugar no ranking de ocorrências: 23
Dezena: 44 | Lugar no ranking de ocorrências: 32
Dezena: 48 | Lugar no ranking de ocorrências: 34
Dezena: 11 | Lugar no ranking de ocorrências: 37
Dezena: 57 | Lugar no ranking de ocorrências: 39
Dezena: 2 | Lugar no ranking de ocorrências: 46
